# 02) 생존분석 및 시각화 (Survival Analysis)

## 실습 목표
1) Clinical만으로 기본 Kaplan-Meier 생존분석
2) mRNA 발현 High/Low 그룹별 생존비교
3) Methylation Hyper/Hypo 그룹별 생존비교
4) 복합 지표 (Expression × Methylation) 생존분석
5) Cox Proportional Hazards 회귀분석


```bash
pip install numpy pandas matplotlib seaborn scikit-learn lifelines
```

## 생존분석 기본 개념
- **생존시간**: 특정 이벤트(사망, 재발 등)가 발생할 때까지의 시간
- **중도절단**: 연구 기간 중 이벤트가 관찰되지 않은 경우
- **Kaplan-Meier**: 비모수적 생존함수 추정 방법
- **Cox 회귀**: 생존시간에 영향을 미치는 요인들을 분석하는 방법

## 필수 라이브러리 임포트
- 생존분석을 위한 lifelines 라이브러리와 통계분석 도구들을 로드합니다.
- Kaplan-Meier 추정, Cox 회귀분석, 통계검정을 위한 모듈들을 준비합니다.

In [ ]:
import os, numpy as np, pandas as pd, seaborn as sns, matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter, CoxPHFitter
from lifelines.statistics import logrank_test
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정 (시스템에 따라 자동 설정)
import platform
system = platform.system()
if system == 'Darwin':  # macOS
    plt.rc('font', family='AppleGothic')
elif system == 'Windows':
    plt.rc('font', family='Malgun Gothic')
else:  # Linux
    import matplotlib.font_manager as fm
    font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
    if os.path.exists(font_path):
        font_prop = fm.FontProperties(fname=font_path)
        plt.rc('font', family=font_prop.get_name())
    else:
        # DejaVu Sans를 fallback으로 사용
        plt.rc('font', family='DejaVu Sans')

# 마이너스 기호 깨짐 방지
plt.rc('axes', unicode_minus=False)

# 시각화 스타일 설정
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)
np.random.seed(42)

print("생존분석 및 시각화")
print("=" * 50)

## 멀티오믹스 데이터 로드
- mRNA, Clinical, Methylation 데이터를 로드하고 공통 샘플로 정렬합니다.
- 생존분석에 필요한 생존시간과 이벤트 정보를 확인합니다.

In [ ]:
print("\n멀티오믹스 데이터 로드 중...")

mrna = pd.read_csv("mrna_matched.txt", sep="\t", index_col=0)
clinical = pd.read_csv("clinical_matched.tsv", sep="\t")
methyl = pd.read_csv("methylation_matched.txt", sep="\t", index_col=0)

# 임상 데이터 컬럼명 정의
SAMPLE_COL   = "Sample ID"
STATUS_COL   = "Overall Survival Status"      # "1:DECEASED" / "0:LIVING"
DURATION_COL = "Overall Survival (Months)"

print(f"원본 데이터 크기:")
print(f"mRNA: {mrna.shape}")
print(f"Clinical: {clinical.shape}")
print(f"Methylation: {methyl.shape}")

# 공통 샘플 정렬
samples = sorted(set(mrna.columns) & set(clinical[SAMPLE_COL]) & set(methyl.columns))

mrna     = mrna.loc[:, samples]
methyl   = methyl.loc[:, samples]
clinical = clinical[clinical[SAMPLE_COL].isin(samples)].reset_index(drop=True)

print("데이터 정렬 완료!")
print(f"공통 샘플: {len(samples)}개")
print(f"최종 데이터: mRNA {mrna.shape}, Clinical {clinical.shape}, Methylation {methyl.shape}")

## 생존분석용 데이터 전처리
- 생존 이벤트와 기간을 분석에 적합한 형태로 변환합니다.
- 결측치가 있는 샘플을 제거하여 분석의 신뢰성을 확보합니다.

In [ ]:
print("\n생존분석용 데이터 전처리 중...")

# 결측치가 있는 행 제거
clinical = clinical.dropna(subset=[DURATION_COL, STATUS_COL]).copy()

# 생존 이벤트와 기간 컬럼 생성
print("이벤트 인코딩 중...")
clinical["event"] = clinical[STATUS_COL].map({"1:DECEASED": 1, "0:LIVING": 0}).astype(int)
clinical["duration"] = clinical[DURATION_COL].astype(float)

# 전처리 결과 확인
n_total = len(clinical)
n_events = clinical["event"].sum()
n_censored = n_total - n_events
median_time = clinical["duration"].median()

print("전처리 완료!")
print(f"분석 대상: {n_total}명")
print(f"이벤트 발생: {n_events}명 ({n_events/n_total*100:.1f}%)")
print(f"중도절단: {n_censored}명 ({n_censored/n_total*100:.1f}%)")
print(f"중앙 생존시간: {median_time:.1f}개월")

### Vibe Coding 미션 C1: 생존데이터 탐색적 분석
**미션**: 생존분석 데이터의 특성을 체계적으로 탐색해보세요!

**도전 과제**:
1. 생존시간의 분포를 히스토그램과 박스플롯으로 시각화하기
2. 이벤트 발생률을 전체적으로 요약하기 (사망/생존 비율)
3. 중도절단된 데이터의 패턴 분석하기


In [ ]:
# 미션 C1 코드를 여기에 작성하세요!


### Vibe Coding 미션 C2: 생존데이터 품질 평가
**미션**: 생존분석 데이터의 품질을 다각도로 평가해보세요!

**도전 과제**:
1. 관찰기간별 이벤트 발생률 트렌드 분석하기
2. 극값(outlier) 생존시간을 가진 환자들 식별하기
3. Follow-up 기간의 적절성 평가하기


In [ ]:
# 미션 C2 코드를 여기에 작성하세요!


## 1) 전체 환자 기본 Kaplan-Meier 생존곡선
- 모든 환자를 대상으로 한 기본적인 생존곡선을 그립니다.
- 전체 코호트의 생존 패턴을 파악하는 기준선 역할을 합니다.

In [ ]:
print("\nKaplan-Meier 생존곡선 생성 중...")

kmf = KaplanMeierFitter()
kmf.fit(clinical["duration"], clinical["event"], label="All Patients")

plt.figure(figsize=(10, 6))
kmf.plot_survival_function(ci_show=True, ci_alpha=0.3)
plt.title("Kaplan-Meier Survival Curve - All Patients", fontsize=14, fontweight='bold')
plt.xlabel("Time (Months)")
plt.ylabel("Survival Probability")
plt.ylim(0, 1)
plt.grid(True, alpha=0.3)

# 중앙 생존시간 표시
median_survival = kmf.median_survival_time_
plt.axvline(median_survival, color='red', linestyle='--', alpha=0.7, label=f'Median: {median_survival:.1f} months')
plt.legend()

plt.tight_layout()
plt.show()

print(f"기본 생존곡선 생성 완료!\n중앙 생존시간: {median_survival:.1f}개월" if not pd.isna(median_survival) else "중앙 생존시간: 도달하지 않음 (Not Reached)")

### Vibe Coding 미션 C3: 기본 생존곡선 향상
**미션**: 기본 생존곡선을 더욱 정보가 풍부하고 보기 좋게 만들어보세요!

**도전 과제**:
1. 생존곡선에 위험군 수 테이블(risk table) 추가하기
2. 특정 시점(12, 24, 36개월)에서의 생존율 표시하기
3. 95% 신뢰구간과 함께 색상 조화로운 테마 적용하기


In [ ]:
# 미션 C3 코드를 여기에 작성하세요!


## 2) mRNA 발현 기반 생존분석
- 특정 유전자의 발현량에 따라 환자를 High/Low 그룹으로 나누어 생존을 비교합니다.
- Log-rank 검정을 통해 그룹 간 생존 차이의 통계적 유의성을 평가합니다.

In [ ]:
print("\nmRNA 발현 기반 생존분석 중...")

# 분석할 유전자 선택 (실습자가 변경 가능)
target_gene = mrna.index[0]   # ← 이 값을 다른 유전자로 바꿔보세요!
print(f"분석 유전자: {target_gene}")

# Clinical 데이터와 유전자 발현 데이터 병합
df = clinical.set_index(SAMPLE_COL).join(mrna.loc[target_gene].rename("Expression"))

# 유전자 발현에 따른 그룹 분류 (중위수 기준)
print("발현 그룹 분류 중...")
df["Expression_group"] = pd.qcut(df["Expression"], 2, labels=["Low", "High"])

# 그룹별 샘플 수 확인
group_counts = df["Expression_group"].value_counts()
print(f"그룹별 환자 수: Low={group_counts['Low']}명, High={group_counts['High']}명")

# Kaplan-Meier 생존곡선 비교
plt.figure(figsize=(10, 6))
kmf = KaplanMeierFitter()
colors = ['#1f77b4', '#ff7f0e']

for i, group in enumerate(["Low", "High"]):
    subset = df[df["Expression_group"] == group]
    kmf.fit(subset["duration"], subset["event"], label=f"{group} Expression")
    kmf.plot_survival_function(ci_show=True, ci_alpha=0.2, color=colors[i])

plt.title(f"Survival Analysis by {target_gene} Expression", fontsize=14, fontweight='bold')
plt.xlabel("Time (Months)")
plt.ylabel("Survival Probability")
plt.ylim(0, 1)
plt.grid(True, alpha=0.3)

# Log-rank 검정 수행
low_group = df[df["Expression_group"] == "Low"]
high_group = df[df["Expression_group"] == "High"]

log_rank_result = logrank_test(
    low_group["duration"], high_group["duration"],
    event_observed_A=low_group["event"], event_observed_B=high_group["event"]
)
p_value = log_rank_result.p_value

# 통계 결과를 플롯에 추가
plt.text(0.02, 0.98, f"Log-rank p-value: {p_value:.4f}", 
         transform=plt.gca().transAxes, fontsize=12,
         bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8),
         verticalalignment='top')

print(f"Log-rank 검정 결과: p = {p_value:.4f}")
print("통계적으로 유의한 생존 차이가 있습니다!" if p_value < 0.05 else "통계적으로 유의한 생존 차이가 없습니다.")

plt.legend()
plt.tight_layout()
plt.show()

print("mRNA 발현 기반 생존분석 완료!")

## 3) Methylation 기반 생존분석
- 메틸화 베타값에 따라 환자를 Hyper/Hypo 그룹으로 나누어 생존을 비교합니다.
- 메틸화와 생존 간의 관계를 탐색합니다.

In [ ]:
print("\nMethylation 기반 생존분석 중...")

# 상위 변이 프로브 선택
print("분석할 프로브 선별 중...")
probe_variance = methyl.var(axis=1, skipna=True)
target_probe = probe_variance.sort_values(ascending=False).index[0]
print(f"선택된 프로브: {target_probe} (분산: {probe_variance[target_probe]:.4f})")

# Clinical 데이터와 메틸화 데이터 병합
df["Methylation"] = methyl.loc[target_probe]
df["Methylation_group"] = pd.qcut(df["Methylation"].dropna(), 2, labels=["Hypo", "Hyper"])

# 결측치 제거
df_meth = df.dropna(subset=["Methylation"])

# 그룹별 샘플 수 확인
meth_counts = df_meth["Methylation_group"].value_counts()
print(f"메틸화 그룹별 환자 수: Hypo={meth_counts.get('Hypo', 0)}명, Hyper={meth_counts.get('Hyper', 0)}명")

# Kaplan-Meier 생존곡선 비교
plt.figure(figsize=(10, 6))
colors = ['#2ca02c', '#d62728'] 

for i, group in enumerate(["Hypo", "Hyper"]):
    subset = df_meth[df_meth["Methylation_group"] == group]
    kmf.fit(subset["duration"], subset["event"], label=f"{group}methylated")
    kmf.plot_survival_function(ci_show=True, ci_alpha=0.2, color=colors[i])

plt.title(f"Survival Analysis by Methylation Status\n(Probe: {target_probe})", 
          fontsize=14, fontweight='bold')
plt.xlabel("Time (Months)")
plt.ylabel("Survival Probability")
plt.ylim(0, 1)
plt.grid(True, alpha=0.3)

# Log-rank 검정 수행
hypo_group = df_meth[df_meth["Methylation_group"] == "Hypo"]
hyper_group = df_meth[df_meth["Methylation_group"] == "Hyper"]

log_rank_result = logrank_test(
    hypo_group["duration"], hyper_group["duration"],
    event_observed_A=hypo_group["event"], event_observed_B=hyper_group["event"]
)
p_value = log_rank_result.p_value

plt.text(0.02, 0.98, f"Log-rank p-value: {p_value:.4f}", 
         transform=plt.gca().transAxes, fontsize=12,
         bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8),
         verticalalignment='top')

print(f"Log-rank 검정 결과: p = {p_value:.4f}")

plt.legend()
plt.tight_layout()
plt.show()

print("Methylation 기반 생존분석 완료!")

## 4) 복합지표 생존분석 (Expression × Methylation)
- 유전자 발현과 메틸화를 동시에 고려한 4개 그룹 생존비교를 수행합니다.
- 멀티오믹스 통합 지표의 예후 예측력을 평가합니다.

In [ ]:
# mRNA와 methylation 그룹을 조합하여 Combined 그룹 생성
df["Combined"] = df["Expression_group"].astype(str) + "_" + df["Methylation_group"].astype(str)

# Kaplan-Meier 생존 곡선
plt.figure(figsize=(8, 6))
kmf = KaplanMeierFitter()

for g, color in zip(df["Combined"].dropna().unique(), ["blue", "red", "green", "orange"]):
    sub = df[df["Combined"] == g]
    kmf.fit(sub["duration"], sub["event"], label=g)
    kmf.plot_survival_function(ci_show=False, color=color)

plt.title("KM by Combined Expression + Methylation Pattern")
plt.xlabel("Time (Months)")
plt.ylabel("Survival Probability")
plt.grid(alpha=0.3)
plt.show()


### Vibe Coding 미션 C4: 복합지표 고급 분석
**미션**: 멀티오믹스 복합지표의 예후 예측력을 체계적으로 평가해보세요!

**도전 과제**:
1. 4개 그룹 간 모든 쌍별(pairwise) 통계 검정 수행하기
2. 복합지표의 예후 예측 성능을 C-index로 평가하기
3. 위험 점수 시스템을 개발하여 환자를 고/중/저 위험군으로 분류하기

In [ ]:
# 미션 C4 코드를 여기에 작성하세요!


## 5) Cox Proportional Hazards 회귀분석
- 연속형 변수를 사용한 다변량 생존분석을 수행합니다.
- Hazard Ratio를 통해 각 요인의 위험도 기여도를 정량화합니다.

In [ ]:
# 1) 필요한 컬럼을 한 번에 준비(없으면 NaN으로 생성)
cox_df = df.reindex(columns=["duration", "event", "Expression", "Methylation"]).copy()

# 2) z-점수 표준화(결측은 각 컬럼 평균→여전히 NaN이면 0으로)
v = pd.to_numeric(cox_df["Expression"], errors="coerce")
cox_df["Expr_z"] = StandardScaler().fit_transform(v.fillna(v.mean()).fillna(0.0).values.reshape(-1,1))

v = pd.to_numeric(cox_df["Methylation"], errors="coerce")
cox_df["Meth_z"] = StandardScaler().fit_transform(v.fillna(v.mean()).fillna(0.0).values.reshape(-1,1))

# 3) Cox 적합 (Expr_z, Meth_z 동시 포함)
cph = CoxPHFitter()
cph.fit(cox_df[["duration","event","Expr_z","Meth_z"]],
        duration_col="duration", event_col="event")
cph.print_summary()


In [ ]:
print("\n   📈 Forest Plot + Baseline Survival Function Subplot 생성 중...")

# Cox Regression Summary
summary = cph.summary
hr = summary["exp(coef)"]
hr_lower = summary["exp(coef) lower 95%"]
hr_upper = summary["exp(coef) upper 95%"]
p_values = summary["p"]

# 변수명 정리
labels = [
    name.replace("_z", "")
        .replace("Expression", "Gene Expression")
        .replace("Methylation", "Methylation Level") 
    for name in hr.index
]

y_pos = np.arange(len(hr))[::-1]

# 1행 2열 subplot
fig, axes = plt.subplots(1, 2, figsize=(18, 7))
fig.suptitle(f"Cox Regression Analysis\nGene: {target_gene}, Probe: {target_probe}", 
             fontsize=16, fontweight='bold')

# ---------------------------------------
# 1. Forest Plot (왼쪽)
# ---------------------------------------
print("   📈 Hazard Ratio Forest Plot 생성 중...")
for pos, hr_val, lower, upper, p_val in zip(y_pos, hr, hr_lower, hr_upper, p_values):
    color = 'red' if p_val < 0.05 else 'steelblue'
    alpha = 0.8 if p_val < 0.05 else 0.6

    # 신뢰구간 에러바
    axes[0].errorbar(
        hr_val, pos,
        xerr=[[hr_val - lower], [upper - hr_val]],
        fmt='o',
        color=color,
        alpha=alpha,
        capsize=5,
        markersize=8,
        linewidth=2
    )

    # HR 값과 p-value 텍스트
    axes[0].text(
        hr_val + 0.02, pos,
        f'HR: {hr_val:.2f}\np: {p_val:.3f}',
        va='center', ha='left', fontsize=9,
        bbox=dict(boxstyle="round,pad=0.2", facecolor="white", alpha=0.7)
    )

# 기준선 (HR = 1)
axes[0].axvline(x=1, color='gray', linestyle='--', alpha=0.8, linewidth=2)

# 축 설정
axes[0].set_yticks(y_pos)
axes[0].set_yticklabels(labels)
axes[0].set_xlabel("Hazard Ratio (95% CI)")
axes[0].set_title("Hazard Ratio Forest Plot", fontsize=14, fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)

# ---------------------------------------
# 2. Baseline Survival Function (오른쪽)
# ---------------------------------------
print("   📈 Baseline 생존함수 시각화 중...")
baseline_survival = cph.baseline_survival_
baseline_survival.plot(ax=axes[1], legend=False, linewidth=2.5, color='darkblue')

axes[1].set_title("Baseline Survival Function S₀(t)", fontsize=14, fontweight='bold')
axes[1].set_xlabel("Time (Months)")
axes[1].set_ylabel("Baseline Survival Probability")
axes[1].set_ylim(0, 1)
axes[1].grid(True, alpha=0.3)

# ---------------------------------------
# 전체 레이아웃 조정
# ---------------------------------------
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

print("   ✅ 분석 완료!")
